# twitterapiv2

This library offers access through Scala to the twitter API v2 endpoints. Additional aids for pagination and throttle management are also provided.

### Set up

In [ ]:
import $ivy.`com.typesafe.akka::akka-http-spray-json:10.2.4`
import $ivy.`com.typesafe.akka::akka-slf4j:2.6.8`
import $ivy.`com.typesafe.akka::akka-actor-typed:2.6.8`
import $ivy.`com.typesafe.akka::akka-stream:2.6.8`
import $ivy.`com.typesafe.akka::akka-http:10.2.4`

Please, create the project `jar` by typing in the sbt prompt:

`sbt:twitterapiv2> package`

In [ ]:
import $cp.target.`scala-2.13`.`twitterapiv2_2.13-0.1.jar`

### Imports and dependencies

In [ ]:
import scala.concurrent.{Future, Await, ExecutionContext, duration}, duration._
import _root_.akka.actor.typed.ActorSystem
import _root_.akka.actor.typed.scaladsl.Behaviors
import scala.util.Success
import scala.util.Failure
import dev.habla.twitter.{v2_akka, v2}, v2_akka._

Common dependencies for actor-based systems and Akka stream:

In [ ]:
implicit val system = ActorSystem(Behaviors.empty, "TwitterV2")
implicit val ec = system.executionContext

Obtain the bearer token from the environment. Remember to pass this variable to docker, in case you started this notebook from there (`$ docker run -e <var_name> ...`)

In [ ]:
def bearerToken = scala.util.Properties.envOrElse("BEARER_TOKEN", "undefined")

### Single requests

We can access programmatically different endpoints to obtain the response of a single request. For instance, we can create a [search/recent](https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-recent) request and obtain its response as follows:

In [ ]:

val response: Future[v2.recents.SingleResponse] = 
    v2.recents.SingleRequest("scala3", bearerToken, max_results=Some(10))
        .single

In [ ]:
response.map{ case v2.recents.Tweets(_, r, s) => (r,s) }

In [ ]:
show(Await.result(response, 1.second))

Similarly, we can [lookup a tweet](https://developer.twitter.com/en/docs/twitter-api/tweets/lookup/api-reference/get-tweets-id) by its identifier as follows:

In [ ]:
Await.result(
    v2.lookupt.Request("787969995917656064", bearerToken)
        .single, 
    Duration.Inf)

In [ ]:
v2.lookupt.Request("787969995917656064", bearerToken)
        .single

### Pagination

In [ ]:
import akka.stream.scaladsl._

But the library can also do pagination and throttle management for us automatically:

In [ ]:
val response: Source[v2.recents.SingleResponse, akka.NotUsed] = 
    v2.recents.SingleRequest("scala3 -is:retweet", bearerToken)
        .stream

Using this stream, we may take the last `N` tweets:

In [ ]:
def takeNTweets(request: v2.recents.SingleRequest)(n: Int): Future[Seq[v2.Tweet]] = 
    request.stream
        .mapConcat{ 
            case v2.recents.Tweets(v2.recents.Tweets.Body(Some(tweets), _, meta), _, _) => 
                tweets 
            case _ => List()
        }.take(n)
        .toMat(Sink.seq)(Keep.right)
        .run


In [ ]:
takeNTweets(v2.recents.SingleRequest("scala3", bearerToken, tweet_fields = Some("id,text,created_at")))(1)